<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Iowa Liquor Sales 

---

Project 3, unlike project 2 which combined skills from weeks 1 and 2, will now be review of week 2. Projects going forward will fit this format: you learn material one week and review it through a project the following week.

That being said, this is as you might expect more challenging. You will be using the pandas data transformation functionality (melt, pivot, groupby, apply) that was not a factor in Project 2.

---

#### Context of the data

The state of Iowa provides many data sets on their website, including [this dataset](https://www.dropbox.com/sh/pf5n5sgfgiri3i8/AACkaMeL_i_WgZ00rpxOOcysa?dl=0) which contains transactions for all stores that have a class E liquor license. You can choose one of the following two scenarios.

The data can also be found [directly on their website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy), which allows you to explore it graphically and download it (though it doesn't work very well).

NOTE: Some of you may have computer issues with the full dataset. In this case, feel free to use [this 10% dataset version of Iowa liquor sales](https://drive.google.com/file/d/0Bx2SHQGVqWaseDB4QU9ZSVFDY2M/view?usp=sharing). You may want to use it anyway to test and prototype your code since it will be faster, before running it on the full dataset.

---

#### Package imports

In [7]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('darkgrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the data from file

---

In [8]:
iowa_file = '/Users/nichollejames/Desktop/DSI-SF-2-oboechick/projects/project-03/Iowa_Liquor_sales_sample_10pct.csv'

iowa = pd.read_csv(iowa_file)

In [9]:
iowa.shape

(270955, 18)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Do an initial overview of the data

---

At the very least describe the columns/variables and the datatypes. 

The column names are Date, Store Number, City, Zip Code, County Number, County, Category, Category Name, Vendor Number, Item Number, Item Description, Bottle Volume (ml), State Bottle Cost, State Bottle Retail, Bottles Sold, Sale (Dollars), Volume Sold (Liters), and Volume Sold (Gallons). The column types are listed below.

        Date                      object 
        Store Number               int64 
        City                      object
        Zip Code                  object
        County Number            float64
        County                    object
        Category                 float64
        Category Name             object
        Vendor Number              int64
        Item Number                int64
        Item Description          object
        Bottle Volume (ml)         int64
        State Bottle Cost         object
        State Bottle Retail       object
        Bottles Sold               int64
        Sale (Dollars)            object
        Volume Sold (Liters)     float64
        Volume Sold (Gallons)    float64


In [10]:
iowa.head(2)

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5,0.40


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Clean the dataset

---

### 3.1 Identify columns that you will need to convert and clean. Where and how is the data corrupted?

Don't worry about converting the date column to a pandas/numpy "datetime" datatype, unless you want to (not required for these problems and is a challenging thing to work with in its own right.)

* Zip Code needs to be looked at
* State Bottle Cost, Sale (Dollars) and State Bottle Retail have '$' that need to be removed before we can do anything with the data.
* Bottle Volume (ml) is an int64 it would probably be easier to deal with if it were a float64. 
* Date is an object, should be date/time.
* County, Category should be int64 not float64

In [11]:
iowa.isnull().sum()

Date                        0
Store Number                0
City                        0
Zip Code                    0
County Number            1077
County                   1077
Category                   68
Category Name             632
Vendor Number               0
Item Number                 0
Item Description            0
Bottle Volume (ml)          0
State Bottle Cost           0
State Bottle Retail         0
Bottles Sold                0
Sale (Dollars)              0
Volume Sold (Liters)        0
Volume Sold (Gallons)       0
dtype: int64

### 3.2 Clean the columns

In [12]:
iowa.columns = [col.replace(" ", "_").replace("(", "").replace(")","").lower() for col in iowa.columns.tolist()]
iowa.columns.tolist()

['date',
 'store_number',
 'city',
 'zip_code',
 'county_number',
 'county',
 'category',
 'category_name',
 'vendor_number',
 'item_number',
 'item_description',
 'bottle_volume_ml',
 'state_bottle_cost',
 'state_bottle_retail',
 'bottles_sold',
 'sale_dollars',
 'volume_sold_liters',
 'volume_sold_gallons']

In [8]:
def price_cleaner(x):
    x = x.replace('$','')
    try:
        return float(x)
    except:
        return np.nan
    
iowa['state_bottle_cost'] = iowa['state_bottle_cost'].map(price_cleaner)
iowa['sale_dollars'] = iowa['sale_dollars'].map(price_cleaner)
iowa['state_bottle_retail'] = iowa['state_bottle_retail'].map(price_cleaner)

iowa.head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,4.50,6.75,12,81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,13.75,20.63,2,41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,12.59,18.89,24,453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,9.50,14.25,6,85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,7.20,10.80,12,129.60,21.0,5.55


In [13]:
iowa['date'] = pd.to_datetime(iowa['date'])

iowa['month'] = iowa['date'].map(lambda val: val.month)
iowa['day'] = iowa['date'].map(lambda val: val.day)
iowa['year'] = iowa['date'].map(lambda val: val.year)



In [14]:
iowa['zip_code'].unique()
iowa[iowa['zip_code'] == '712-2']


,date,store_number,city,zip_code,county_number,county,category,category_name,vendor_number,item_number,...,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons,month,day,year
114,2015-01-16,4307,DUNLAP,712-2,43.0,Harrison,1062300.0,FLAVORED RUM,35,43136,...,750,$8.50,$12.75,2,$25.50,1.50,0.40,1,16,2015
2695,2015-01-05,4307,DUNLAP,712-2,43.0,Harrison,1012100.0,CANADIAN WHISKIES,115,11777,...,1000,$6.63,$9.95,12,$119.40,12.00,3.17,1,5,2015
4129,2015-10-19,4307,DUNLAP,712-2,43.0,Harrison,1012100.0,CANADIAN WHISKIES,115,10553,...,750,$6.75,$10.13,3,$30.39,2.25,0.59,10,19,2015
5506,2015-01-05,4307,DUNLAP,712-2,43.0,Harrison,1081380.0,MISCELLANEOUS SCHNAPPS,65,82627,...,1000,$7.62,$11.43,1,$11.43,1.00,0.26,1,5,2015
5512,2015-11-20,4307,DUNLAP,712-2,43.0,Harrison,1051140.0,PEACH BRANDIES,434,56206,...,750,$5.48,$8.22,1,$8.22,0.75,0.20,11,20,2015
8186,2016-03-14,4307,DUNLAP,712-2,NaN,NaN,1031080.0,VODKA 80 PROOF,55,35314,...,375,$1.71,$2.57,3,$7.71,1.12,0.30,3,14,2016
8642,2015-11-02,4307,DUNLAP,712-2,43.0,Harrison,1081400.0,TRIPLE SEC,434,86310,...,1000,$3.00,$4.50,1,$4.50,1.00,0.26,11,2,2015
9347,2016-03-21,4307,DUNLAP,712-2,NaN,NaN,1081400.0,TRIPLE SEC,434,86310,...,1000,$3.00,$4.50,3,$13.50,3.00,0.79,3,21,2016
16502,2015-12-26,4307,DUNLAP,712-2,43.0,Harrison,1031200.0,VODKA FLAVORED,380,41783,...,500,$4.89,$7.34,1,$7.34,0.50,0.13,12,26,2015
16928,2015-11-09,4307,DUNLAP,712-2,43.0,Harrison,1062300.0,FLAVORED RUM,260,43287,...,750,$7.49,$11.24,1,$11.24,0.75,0.20,11,9,2015


In [15]:
iowa['zip_code'].replace(51529,'51529',inplace=True)

In [16]:
iowa.head(10).dtypes

date                   datetime64[ns]
store_number                    int64
city                           object
zip_code                       object
county_number                 float64
county                         object
category                      float64
category_name                  object
vendor_number                   int64
item_number                     int64
item_description               object
bottle_volume_ml                int64
state_bottle_cost              object
state_bottle_retail            object
bottles_sold                    int64
sale_dollars                   object
volume_sold_liters            float64
volume_sold_gallons           float64
month                           int64
day                             int64
year                            int64
dtype: object

In [17]:
def int_to_float(x):
    if type(x) == int:
        return float(x)
    
iowa['bottle_volume_ml'] = iowa['bottle_volume_ml'].map(int_to_float)
iowa['bottles_sold'] = iowa['bottles_sold'].map(int_to_float)

In [18]:
def int_to_str(x):
    if type(x) == int:
        return str(x)

iowa['store_number'] = iowa['store_number'].map(int_to_str)
#iowa['Category'] = iowa['Category'].map(float_to_int)

iowa.dtypes

date                   datetime64[ns]
store_number                   object
city                           object
zip_code                       object
county_number                 float64
county                         object
category                      float64
category_name                  object
vendor_number                   int64
item_number                     int64
item_description               object
bottle_volume_ml              float64
state_bottle_cost              object
state_bottle_retail            object
bottles_sold                  float64
sale_dollars                   object
volume_sold_liters            float64
volume_sold_gallons           float64
month                           int64
day                             int64
year                            int64
dtype: object

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 3.3 Perform more extensive cleaning of the dataset

Cleaning of data can mean a lot more than just fixing strings and numbers in columns. There are often logical errors with data, useless or nonsensical categories, redundancy of information, outliers, and many more problems.

This dataset has problems beyond just fixing the types of columns. Though resolving them may not be required for EDA and analysis, if you want experience with "deeper" cleaning of data this is a great dataset to start practicing with.

Keep in mind that some types of "data cleaning" is subjective: it's not always a cut-and-dry conversion of type or removal of null values. Subjectivity when dealing with data is just a fact of life for a data scientist. This isn't a kind of programming where things are just right or wrong.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Filter/adjust the store data

---

Some stores may have opened or closed in 2015. These stores will have incorrect yearly summary statistics since they were not open the full year. We need to filter them out or find another way to deal with the inconsistent numbers of months across stores.

It is up to you how you want to deal with this problem.

1. Investigate problematic stores.
2. Decide on an approach to deal with stores not open for the full 2015 year.
3. Implement your plan.
4. Briefly report on what you did and why.


In [ ]:
mask1 = store_name have sales post 2015
        
mask2 = store_name have sales pre 2015
        groupby sales in mask2

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Proposals for new tax rates (Scenario 1)

---

You are a data scientist in residence at the Iowa State tax board. The Iowa State legislature is considering changes in the liquor tax rates and has assigned you to the project.

### 5.1 Calculate yearly liquor sales for each store in 2015.

In [15]:
iowa.head(2)

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,...,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),MONTH,DAY,YEAR
0,2015-11-04,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,...,750.0,4.50,6.75,12.0,81.00,9.0,2.38,11,4,2015
1,2016-03-02,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,...,750.0,13.75,20.63,2.0,41.26,1.5,0.40,3,2,2016


In [53]:
def yearly_liquor_sales(x):
    store = []
    sales = []
    for store_number in x:
        if store_number == x:
            print '-'
            if date in iowa['date'] >= '01/01/2015' and iowa['date'] < '01/01/2016':
                print '--'
                for sale in iowa['sale_dollars']:
                    sales.append(sale)
                    return sales
            print '--------------'
            print 'Store Number: ', store_number
            print 'Total Sales 2015: ', np.sum(sales)
total_per_store = iowa['store_number'].apply(yearly_liquor_sales)
total_per_store

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
270925    None
270926    None
270927    None
270928    None
270929    None
270930    None
270931    None
270932    None
270933    None
270934    None
270935    None
270936    None
270937    None
270938    None
270939    None
270940    None
270941    None
270942    None
270943    None
270944    None
270945    None
270946    None
270947    None
270948    None
270949    None
270950    None
270951    None
270952    None
270953    None
270954    None
Name: store_number, dtype: object

### 5.2 Calculate the profit each store is making in 2015.


In [ ]:
ran out of time.

### 5.3 Investigate which Iowa counties are making the most profit on liquor per gallon in 2015.

In [ ]:
ran out of time

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.4 Plot a color map of average profit per county in 2015.

[Check out this site](http://flowingdata.com/2009/11/12/how-to-make-a-us-county-thematic-map-using-free-tools/) for  information on how to create map visualizations in python. There are other resources online for map plotting in python as well.

Another interesting map plot might show the difference in profit between time periods. Change in profit from November to December, for example.

### 5.5  Does profit scale proportionally to store size?

Size of a store can be defined as the total gallons sold in 2015. Is the profit to store size ratio increasing as store size increases? Decreasing? Consistent?

In [ ]:
ran out of time

### 5.6 Create a broader category for liquor type.

Liquor types are pretty granular in this dataset. Create a column that categorizes these types into a smaller amount of categories. The categories you decide on are up to you.

In [ ]:
ran out of time

### 5.7 Do relationships exist between the broader liquor type and profit?

In [ ]:
ran out of time

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.8 Are there any noticeable relationships between liquor brand and sales? Profit?

### 5.9 The tax board wants to design a tax or taxes that affect larger stores more than smaller "mom and pop" stores.

Based on your investigations into the data, come up with a way you could design a tax that achieves this goal **without explicitly taxing stores based on size or county critera.** The liqour board does not want to obviously punish larger stores or speific counties for fear of backlash, but is willing to tax hard alcohol more than beer, for example.

Feel free to do more EDA if it helps.

Your report should describe whether such a tax is possible or not, and the specifics of what the tax will target/do.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Market research for new store locations (Scenario 2)

A liquor store owner in Iowa is looking to open a store in a new location and has hired you to investigate the market data for potential new locations. The business owner is interested in what you can discover from the data that will help him evaluate a good location for a new storefront.

---

### 6.1 Create columns for Q1 2015 sales and Q1 2016 sales.

The owner is not just interested in what locations are currently selling well, but also which areas are showing the most promising sales growth from 2015 to 2016. 

For yearly changes we will focus on the changes from Q1 2015 to Q1 2016 specifically (avoiding effects of season changes).

Note: Q1 is months: January, February, and March.


In [ ]:
ran out of time 

### 6.2 Decide on a metric for performance that combines Q1 2016 sales as well as growth in sales from Q1 2015.

Again, the owner wants to know that his store will be in a good position to sell a large amount of liquor now, but also have his store be more likely to grow and sell even more going forward. 

In particular, he is worried about opening a store in an area that is showing a decline in liquor sales between Q1 2015 and Q1 2016.

In [ ]:
ran out of time 

### 6.3 Recommend a location based on your metric.

The owner asks you to make a recommendation for a new store location. Explain why you recommended this area.

In [ ]:
ran out of time

### 6.4 Recommend a location based also on a restricted inventory type.

Your employer has now decided he wants his store to focus on one type of liquor only. He askes you:

1. What would be the best type of liquor to sell to maximize profit?
2. Should the recommended location change if he is only selling one type of liquor?

In [ ]:
ran out of time

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Time-related effects

---

You could imagine that liquor sales might be affected by a variety of effects related to time. Do people buy more beer in the summer? Do liquor sales skyrocket in december? Do people buy less liquor on Tuesdays?

You have the date of sales in your dataset, which you can use to pull out time components.

1. Come up with 3 different hypotheses about how liquor sales may vary with time-related variables. 
2. Create a visualization exploring each hypothesis.
3. Write brief concluding remarks on what you observed.


<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Appropriate spending

---

The owner you worked with before realized that opening new stores cost money, something he had completely overlooked earlier. He brings you back in to help him figure out how much he should be willing to spend on opening his store.

### 8.1 Calculate costs and time to recoup.

The owner has decided he will buy 20,000 gallons of the liquor type you recommended (randomly sampled across the different brands/subtypes). He will also be opening his store in the area you recommended.

The cost of buying the property for his store will be $150,000.

1. How much should he expect to spend on the 20,000 gallons of liquor alone?
2. Based on the amount he's spent on the liquor and property, and on the profit per time period he is expected to have, how long will it take him to recoup the costs?

<img src="http://imgur.com/gGrau8s.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">


### 8.2 Calculate maximum loan interest rate.

The owner unfortunately has to take out a loan to to pay for the entirety of the property and liquor. 

**Conditions of the loan**:

Every quarter, (except for Q1 when he takes out the loan), the interest rate is applied to the remaining balance on the loan and added to his debt. 

If, for example, he made no payments whatsoever, not only would the amount of debt he owed keep increasing by quarter but the debt incurred from interest would keep increasing as well. (Since his debt keeps growing, the interest rate is applied each month to the bigger balance, incurring more debt each time.)

**Determining maximum acceptable interest:**

Given this, _the owner wants to be 99% confident that he will be able to pay off his loans._ If the interest rate is too high he will have to eventually declare bankruptcy.

Based on his expected profits per quarter (assume the expected profit stays the same for all future quarters), what is the maximum interest rate he should accept on the loan to be 99% confident he will be able to pay it off?